# sxs_catalog_download_example

This notebook demonstrates how to use the `sxs` python library to download data from the SXS Catalog of waveforms hosted on Zenodo (https://github.com/moble/sxs). The catalog is available at https://black-holes.org/waveforms and is described in https://arxiv.org/abs/1904.04831.

You can install `sxs` via `pip install sxs`. You'll need version `2019.4.15.16.32.10` or later.

Note: You should use a recent version of python 3, such as 3.6.5, to run this notebook. I installed python 3.6.5 using anaconda (https://www.anaconda.com). 

Note: I found I had to also install tqdm, which you can install similarly. This should be installed when you install the `sxs` library with `pip`, but in case you see errors about being unable to import `tqdm`, `pip install tqdm` solves this.

## How to download data

This section demonstrates how to download simulation data from Zenodo. You can download data from one simulation or from multiple simulations at once.

In [ ]:
# For downloading data
import sxs
from sxs import zenodo as zen

# For interacting with the data
import h5py
import numpy as np
from matplotlib import pyplot as plt
import json

This line attempts to download select files from a specific simulation. 

`dry_run = True` means that the function does everything except actually download the data; set this to false to download the data into the same directory as this notebook.

You can download dat for multiple simulations by changing `sxs_ids` to include more simulations, or to include an expression that matches multiple simulations. For instance, `sxs_ids = ['SXS:BBH:']` would download all binary-black-hole waveforms from the catalog.

Set `highest_lev_only = True` to download only the highest resoution of each file that is available at multiple resolutions, instead of downloading files from all resolutions.

In [ ]:
zen.download.matching("common-metadata.txt", "metadata.txt", \
                      "rhOverM_Asymptotic_GeometricUnits_CoM.h5", \
                      "Horizons.h5", \
                      sxs_ids = ['SXS:BBH:0444'], \
                      dry_run = True, \
                      highest_lev_only = False)

Print the help text for a function that downloads data from the catalog, for more details on how the function works.

In [ ]:
?zen.download.matching

## Examples of using data from the catalog

Download the highest resolution Horizons and waveform files for a specific simulation. This will download the data in the same notebook as this notebook, overwriting other files of the same name if they exist.

In [ ]:
zen.download.matching("common-metadata.txt", "metadata.txt", "metadata.json", \
                      "rhOverM_Asymptotic_GeometricUnits_CoM.h5", \
                      "Horizons.h5", \
                      sxs_ids = ['SXS:BBH:0444'], \
                      dry_run = False, \
                      highest_lev_only = True)

In [ ]:
horizons = h5py.File("SXS_BBH_0444/Horizons.h5", 'r')
rhOverM = h5py.File("SXS_BBH_0444/rhOverM_Asymptotic_GeometricUnits_CoM.h5", 'r')

Keys of the waveform file specify the extrapolation order or a text file giving version history.

In [ ]:
print(list(rhOverM.keys()))
print(np.array(rhOverM['VersionHist.ver']))

Each extrapolation order has some metadata given as attributes.

In [ ]:
print(list(rhOverM['Extrapolated_N2.dir'].attrs.keys()))
for key in rhOverM['Extrapolated_N2.dir'].attrs.keys():
    print(key + " = " + str(rhOverM['Extrapolated_N2.dir'].attrs[key]))

Here's an example that plots waveform data from the downloaded H5 files.

In [ ]:
timeReIm = np.array(rhOverM['Extrapolated_N2.dir']['Y_l2_m2.dat'])
plt.clf()
plt.plot(timeReIm[:,0], timeReIm[:,1])
plt.xlabel("Time (code units)")
plt.ylabel("Strain")
plt.show()

Here's a similar example plotting the irreducible mass vs. time.

In [ ]:
list(horizons.keys())

In [ ]:
list(horizons['AhA.dir'].keys())

In [ ]:
plt.clf()
plt.plot(horizons['AhA.dir']['ArealMass.dat'][:,0], horizons['AhA.dir']['ArealMass.dat'][:,1], label='AhA')
plt.plot(horizons['AhB.dir']['ArealMass.dat'][:,0], horizons['AhB.dir']['ArealMass.dat'][:,1], label='AhB')
plt.plot(horizons['AhC.dir']['ArealMass.dat'][:,0], horizons['AhC.dir']['ArealMass.dat'][:,1], label='AhC')
plt.xlabel("Time (code units)")
plt.ylabel("Irreducible mass (code units)")
plt.show()

## Example of using metadata in the catalog

In [ ]:
with open("SXS_BBH_0444/metadata.json") as file:
    metadata_json = json.load(file)

Here is code to parse the metadata json file `metadata.json`.

In [ ]:
list(metadata_json.keys())

In [ ]:
metadata_json['remnant_mass']

And here is an example getting the same metadata from `metadata.txt`.

In [ ]:
with open("SXS_BBH_0444/metadata.json") as file:
    metadata_txt_raw = file.readlines()

In [ ]:
# Simple code to kind of parse the text file. Room for improvement.
metadata_txt = {}
for line in metadata_txt_raw:
    if ':' in line:
        key = line.split('":')[0].strip().split('"')[1]
        value = line.split('":')[-1].split(',')[0].strip()
        metadata_txt[key] = value

In [ ]:
float(metadata_txt['remnant_mass'])